# Interactive TinyMPC Example in Python (extended version)
We demonstrate an interactive workflow in Python with TinyMPC where you can generate C code and interact with it via Python. This example shows the entire workflow from quadrotor nonlinear dynamics

Restart kernel if something breaks.

Load necessary packages, make sure to install `tinympc` ([README.md](../README.md))

In [ ]:
import tinympc
import math

Define the class and compile original TinyMPC code to get a generic shared/dynamic library

**PLEASE CHANGE `tinympc_python_dir` TO YOUR ABSOLUTE PATH**

In [ ]:
prob = tinympc.TinyMPC()

tinympc_python_dir = "/home/khai/SSD/Code/tinympc-python"  # Your absolute path to the tinympc-python directory, you may only need to change this

tinympc_dir = tinympc_python_dir + "/tinympc/TinyMPC"  # Path to the TinyMPC directory (C code)
prob.compile_lib(tinympc_dir)  # Compile the library

Load the generic shared/dynamic library 

In [ ]:
lib_dir = tinympc_dir + "/build/src/tinympc/libtinympcShared.so"
prob.load_lib(lib_dir)  # Load the library

We start by building the quadrotor nonlinear dynamics and linearize around the hovering position. Try LQR to see if it work!

In [ ]:
import autograd.numpy as np
import autograd.numpy as sqrt
from autograd.numpy.linalg import norm
from autograd.numpy.linalg import inv
from autograd import jacobian
from autograd.test_util import check_grads
np.set_printoptions(precision=3, suppress=True)

# autograd does not work with np.block

#Quaternion stuff, check `Planning with Attitude` paper for more details
def hat(v):
    return np.array([[0, -v[2], v[1]],
                     [v[2], 0, -v[0]],
                     [-v[1], v[0], 0.0]])

def L(q):
    s = q[0]
    v = q[1:4]
    up = np.hstack([s, -v])
    down = np.hstack([v.reshape(3,1), s*np.eye(3) + hat(v)])
    L = np.vstack([up,down])
    return L

T = np.diag([1.0, -1, -1, -1])
H = np.vstack([np.zeros((1,3)), np.eye(3)])

def qtoQ(q):
    return H.T @ T @ L(q) @ T @ L(q) @ H

def G(q):
    return L(q) @ H

def rptoq(phi):
    return (1./sqrt(1+phi.T @ phi)) * np.hstack([1, phi])

def qtorp(q):
    return q[1:4]/q[0]

def E(q):
    up = np.hstack([np.eye(3), np.zeros((3,3)), np.zeros((3,6))])
    mid = np.hstack([np.zeros((4,3)), G(q), np.zeros((4,6))])
    down = np.hstack([np.zeros((6,3)), np.zeros((6,3)), np.eye(6)])
    E = np.vstack([up, mid, down])
    return E

# Quadrotor parameters
m = 0.035  # mass
J = np.array([[1.66e-5, 0.83e-6, 0.72e-6], [0.83e-6, 1.66e-5, 1.8e-6], [0.72e-6, 1.8e-6, 2.93e-5]])  # inertia
g = 9.81  # gravity
# thrustToTorque = 0.005964552
thrustToTorque = 0.0008  # thrust to torque ratio
el = 0.046/1.414213562  # arm length
scale = 65535  # PWM scale
kt = 2.245365e-6*scale # thrust coefficient, u is PWM in range [0...1]
km = kt*thrustToTorque # moment coefficient

freq = 50.0 # control frequency
h = 1/freq #50 Hz 

Nx1 = 13        # number of states (quaternion)
Nx = 12         # number of states (linearized): x, y, z, Rodriguez 3-parameters (p, q, r), vx, vy, vz, wx, wy, wz
Nu = 4          # number of controls (motor pwm signals, 0-1)
nx = Nx1
nu = Nu
Tfinal = 5.0 # final time
Nt = int(Tfinal/h)+1    # number of time steps
t_vec = np.linspace(0, Tfinal, Nt)
thist = t_vec


In [ ]:
# Quadrotor dynamics
def quad_dynamics(x, u):
    r = x[0:3]
    q = x[3:7]/norm(x[3:7])
    v = x[7:10]
    omg = x[10:13]
    Q = qtoQ(q)

    dr = v
    dq = 0.5*L(q)@H@omg
    dv = np.array([0, 0, -g]) + (1/m)*Q@np.array([[0, 0, 0, 0], [0, 0, 0, 0], [kt, kt, kt, kt]])@u
    domg = inv(J)@(-hat(omg)@J@omg + np.array([[-el*kt, -el*kt, el*kt, el*kt], [-el*kt, el*kt, el*kt, -el*kt], [-km, km, -km, km]])@u)
    
    return np.hstack([dr, dq, dv, domg])

# RK4 integration with zero-order hold on u
def quad_dynamics_rk4(x, u):
    #RK4 integration with zero-order hold on u
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5*h*f1, u)
    f3 = quad_dynamics(x + 0.5*h*f2, u)
    f4 = quad_dynamics(x + h*f3, u)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    xnormalized = xn[3:7]/norm(xn[3:7]) 
    return np.hstack([xn[0:3], xnormalized, xn[7:13]])

# Linearize the dynamics around x0, u0
A_jac = jacobian(quad_dynamics_rk4, 0)  # jacobian wrt x
B_jac = jacobian(quad_dynamics_rk4, 1)  # jacobian wrt u

rg = np.array([0.0, 0, 0.0])
qg = np.array([1.0, 0, 0, 0])
vg = np.zeros(3)
omgg = np.zeros(3)
x0 = np.hstack([rg, qg, vg, omgg])
u0 = (m*g/kt/4)*np.ones(4)
print(x0, u0)

check_grads(quad_dynamics_rk4, modes=['rev'], order=2)(x0, u0)

Anp1 = A_jac(x0, u0)  # jacobian of the dynamics wrt x at x0, u0
Bnp1 = B_jac(x0, u0)  # jacobian of the dynamics wrt u at x0, u0

# `Planning with Attitude` trick, attitude Jacobians
Anp = E(qg).T @ Anp1 @ E(qg)
Bnp = E(qg).T @ Bnp1
print("A = \n", Anp)
print("B = \n", Bnp)

x_all = []
# simulate the dynamics with the zero controller
# x = np.array([0, 0.1, 0, 0])
# for i in range(300):
#     x = cartpole_dynamics_rk4(x, u0)
#     x_all.append(x)

In [ ]:
# Riccati recursion on the linearized dynamics
max_dev_x = np.array([0.1, 0.1, 0.1,  0.5, 0.5, 0.03,  0.5, 0.5, 0.5,  0.7, 0.7, 0.2])
max_dev_u = np.array([0.5, 0.5, 0.5, 0.5])/6
Q = np.diag(1./max_dev_x**2)
R = np.diag(1./max_dev_u**2)
# print("Q = \n", Q)
# print("R = \n", R)

P = Q
for i in range(100):
    P = Q + Anp.T @ P @ Anp - Anp.T @ P @ Bnp @ inv(R + Bnp.T @ P @ Bnp) @ Bnp.T @ P @ Anp
    # print(P)
    K = inv(R + Bnp.T @ P @ Bnp) @ Bnp.T @ P @ Anp
# print("K = \n", K)

# Drive the system from initial state to the hovering state using the LQR controller:
# LQR controller, input is 13x1 state vector
def lqr_controller(x):
    q = x[3:7]
    phi = qtorp(L(qg).T @ q)
    delta_x = np.hstack([x[0:3]-rg, phi, x[7:10]-vg, x[10:13]-omgg])
    return -K @ delta_x + u0

x = x0
x[0:3] = rg + 1*np.random.randn(3)/3  # initial position
NSIM = Nt
x_all = []
# simulate the dynamics with the LQR controller
for i in range(NSIM):
    u = lqr_controller(x)
    x = quad_dynamics_rk4(x, u)
    x_all.append(x)
x_all
# Go to the visualization part and run it to see the trajectory `x_all`

Here we setup problem data and settings for TinyMPC

In [ ]:
n = 12
m = 4
N = 10

A = Anp.transpose().reshape((n * n)).tolist()
B = Bnp.reshape((n * m)).tolist()
print(A)
print(B)
rho = 0.1

x_min = [-5.0] * n * N  # state constraints
x_max = [5.] * n * N  # state constraints
u_min = [-0.5] * m * (N - 1)  # force constraints
u_max = [0.5] * m * (N - 1)  # force constraints

abs_pri_tol = 1.0e-3  # absolute primal tolerance
abs_dual_tol = 1.0e-3  # absolute dual tolerance
max_iter = 100  # maximum number of iterations
check_termination = 1  # whether to check termination and period

# Setup problem data
prob.setup(n, m, N, A, B, Q, R, x_min, x_max, u_min, u_max, rho, abs_pri_tol, abs_dual_tol, max_iter, check_termination)

After define the problem, we generate the tailored code with above data. 

Here we compile it for interactive Python script but you can use it directly for your applications/systems

In [ ]:
output_dir = tinympc_python_dir + "/generated_code1"  # Path to the generated code
prob.tiny_codegen(tinympc_dir, output_dir)
prob.compile_lib(output_dir)

I try to put all of these into a local scope but they are still persistent within a kernel session.

In [ ]:
x_all = []  # List of all stored states

def mpc_tasks():
    solver = tinympc.TinyMPC()
    solver.setup(n, m, N, A, B, Q, R, x_min, x_max, u_min, u_max, rho, abs_pri_tol, abs_dual_tol, max_iter, check_termination)

    solver.load_lib(output_dir + "/build/tinympc/libtinympcShared.so")  
    x = [0.5, 0, -0.4 + math.pi, 0.1] # Initial state
    u = [0.0] * m * (N - 1)  # List of control inputs in horizon

    delta_x_noise = (x - x0) * 1

    # Use delta because MPC uses the linearized dynamics around upright position
    # delta_xref_next = [1.0, 0.0, 0.0, 0.0] * N  # List of reference states in horizon, col-major

    # Set the reference state to 0 as well as reset
    delta_xref = np.repeat(np.array([[0.0], [0.0], [0.0], [0.0]]), N, 1).transpose().reshape(n*N).tolist()
    # solver.set_xref(delta_xref, 1)  # Set the reference state to C code

    # Set the reference x to 1 at step 200
    delta_xref_next = np.repeat(np.array([[1.0], [0.0], [0.0], [0.0]]), N, 1).transpose().reshape(n*N).tolist()

    print("=== START INTERACTIVE MPC ===")

    NSIM = 400
    for i in range(NSIM):
        # 1. Set initial state from measurement    
        solver.set_x0(delta_x_noise, 0)  # Set initial state to C code
        
        # 2. Set the reference state if needed
        # At step 200, set x = 1
        if (i == 200):
            solver.set_xref(delta_xref_next, 0)  # Set the reference state to C code

        # 3. Solve the problem
        solver.solve(0)  # Call the solve in C code

        # 4. Get the control input
        solver.get_u(u, 0)  # Get the control input from C code

        # 5. Simulate the dynamics
        x = quad_dynamics_rk4(np.array(x).reshape((n, 1)), u)

        noise = np.random.normal(0, 0.01, (n, 1))
        delta_x_noise = (x + noise.reshape((n, 1)) - x0.reshape((n,1))).reshape(n).tolist()
        x = x.reshape(n).tolist()
        x_all.append(x)

    print(len(x_all))

mpc_tasks()

Run the interactive MPC example which calls the generated code, use real nonlinear dynamics

This works with pointers then C data is modified in each kernel session. If you want to run from the original setup, you may need to change data back or 
restart kernel.

Visualize the robot and trajectory

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Set up the figure and axis for plotting
fig, ax = plt.subplots(2, 1)

# Plot the trajectory
x_all = np.array(x_all)
ax[0].plot(thist, x_all[:, 0], label="x", linewidth=2)
ax[0].plot(thist, x_all[:, 1], label="y", linewidth=2)
ax[0].plot(thist, x_all[:, 2], label="z", linewidth=2)
ax[0].legend()
ax[0].title.set_text("Position")

ax[1].plot(thist, x_all[:, 3], label="q0", linewidth=2)
ax[1].plot(thist, x_all[:, 4], label="q1", linewidth=2)
ax[1].plot(thist, x_all[:, 5], label="q2", linewidth=2)
ax[1].plot(thist, x_all[:, 6], label="q3", linewidth=2)
ax[1].legend()
ax[1].title.set_text("Attitude")
plt.show()

Save the animation

In [ ]:
ani.save('cartpole3.mp4', writer='ffmpeg', fps=30, dpi=200)

After testing MPC procedure with the generated code, you need to deploy it for your applications/systems. Stay tuned for Teensy and STM32 deployment tutorials. 

Your `tiny_main` may look like this

```C
int main()
{
    int exitflag = 1;
    TinyWorkspace* work = tiny_data_solver.work;
    tiny_data_solver.work->Xref = tiny_MatrixNxNh::Zero();
    tiny_data_solver.work->Uref = tiny_MatrixNuNhm1::Zero();
    tiny_data_solver.settings->max_iter = 150;
    tiny_data_solver.settings->en_input_bound = 1;
    tiny_data_solver.settings->en_state_bound = 1;

    tiny_VectorNx x0, x1; // current and next simulation states
    x0 << 0.0, 0, 0.1, 0; // initial state

    int i = 0;
    for (int k = 0; k < 300; ++k)
    {
        printf("tracking error at step %2d: %.4f\n", k, (x0 - work->Xref.col(1)).norm());

        // 1. Update measurement
        work->x.col(0) = x0;

        // 2. Update reference (if needed)
        // you can also use C wrapper (intended for high-level languages) 
        // by including tiny_wrapper.hpp and call `set_xref(...)` function

        // 3. Reset dual variables (if needed)
        work->y = tiny_MatrixNuNhm1::Zero();
        work->g = tiny_MatrixNxNh::Zero();

        // 4. Solve MPC problem
        exitflag = tiny_solve(&tiny_data_solver);

        // if (exitflag == 0)
        // 	printf("HOORAY! Solved with no error!\n");
        // else
        // 	printf("OOPS! Something went wrong!\n");
        // 	// break;

        std::cout << work->iter << std::endl;
        std::cout << work->u.col(0).transpose().format(CleanFmt) << std::endl;

        // 5. Simulate forward
        // work->u.col(0) = -tiny_data_solver.cache->Kinf * (x0 - work->Xref.col(0));
        x1 = work->Adyn * x0 + work->Bdyn * work->u.col(0);
        x0 = x1;
        // std::cout << x0.transpose().format(CleanFmt) << std::endl;
    }
}
```